In [2]:
from apiclient.discovery import build
from apiclient.errors import HttpError
import pandas as pd
import json
# from oauth2client.tools import argparser

- aaa

In [3]:
# Set DEVELOPER_KEY to the API key value from the APIs & auth > Registered apps
# tab of
#   https://cloud.google.com/console
# Please ensure that you have enabled the YouTube Data API for your project.
with open('./secret.json') as f:
    DEVELOPER_KEY = json.load(f)['APIKEY']
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

In [ ]:
# item = search_response['items'][0]
items_id = []
for item in search_response['items']:
    items_id.append({
        'video_id': item['id']['videoId'], 
        'channel_id': item['snippet']['channelId']
        })

In [ ]:
df = pd.DataFrame(items_id)
df.head()

,video_id,channel_id
0,s8XjEuplx_U,UC8butISFwT-Wl7EV0hUK0BQ
1,bsq2YY-XAEo,UC0xRMqPOyRNPTaL6BxhbCnQ
2,PXMJ6FS7llk,UC8butISFwT-Wl7EV0hUK0BQ
3,h0tYyIY0T2Y,UCMMjv61LfBy5J3AT8Ua0NGQ
4,-5v7CuXm7Ns,UCGlgXjYVoHLD86TQQ799WIw


# 動画検索

In [ ]:
class Youtube:
    def __init__(self, service: str, version: str, developerkey: str) -> None:
        self.app = build(service, version, developerKey=developerkey)
    
    def search(self, keyword: str, max_results: int=50) -> pd.DataFrame:
        search_response = self.app.search().list(
            q=keyword,
            part="id,snippet",
            order='viewCount',
            type='video',
            maxResults=max_results
        ).execute()

        items_id = []
        for item in search_response['items']:
            items_id.append({
                'video_id': item['id']['videoId'], 
                'channel_id': item['snippet']['channelId']
                })

        df = pd.DataFrame(items_id)
        return df

In [ ]:
app = Youtube(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, DEVELOPER_KEY)
df_video = app.search('Python 自動化')
df_video.head()

,video_id,channel_id
0,s8XjEuplx_U,UC8butISFwT-Wl7EV0hUK0BQ
1,bsq2YY-XAEo,UC0xRMqPOyRNPTaL6BxhbCnQ
2,PXMJ6FS7llk,UC8butISFwT-Wl7EV0hUK0BQ
3,h0tYyIY0T2Y,UCMMjv61LfBy5J3AT8Ua0NGQ
4,-5v7CuXm7Ns,UCGlgXjYVoHLD86TQQ799WIw


In [ ]:
channles_id = df_video['channel_id'].unique().tolist()
channles_id[:3]

['UC8butISFwT-Wl7EV0hUK0BQ',
 'UC0xRMqPOyRNPTaL6BxhbCnQ',
 'UCMMjv61LfBy5J3AT8Ua0NGQ']

In [ ]:
statistics_list = youtube.channels().list(
    id=','.join(channles_id),
    part='statistics',
    fields='items(id, statistics(subscriberCount))',
    maxResults=50
).execute()
statistics_list['items'][:3]

[{'id': 'UC0xRMqPOyRNPTaL6BxhbCnQ',
  'statistics': {'subscriberCount': '139000'}},
 {'id': 'UCGFxqqRaM97VjM5ToRfkoFg', 'statistics': {'subscriberCount': '8760'}},
 {'id': 'UC452hry4DI_pDn_CVT9ZKhQ',
  'statistics': {'subscriberCount': '99200'}}]

# 統計情報取得

In [ ]:
statistics = []
for item in statistics_list['items']:
    statistic = {
        'channel_id': item['id'],
        # 'viewCount': int(item['statistics']['viewCount']),
        'subscriberCount': int(item['statistics']['subscriberCount'])
    }
    statistics.append(statistic)

statistics_df = pd.DataFrame(statistics)
statistics_df.head()
    

,channel_id,subscriberCount
0,UC0xRMqPOyRNPTaL6BxhbCnQ,139000
1,UCGFxqqRaM97VjM5ToRfkoFg,8760
2,UC452hry4DI_pDn_CVT9ZKhQ,99200
3,UC5Kgc_HNzx4GJ-w4QMeeKiQ,56300
4,UC7EVSn5inapL20oPSwAwEUg,15700


In [ ]:
merge_df = pd.merge(left=df_video, right=statistics_df, on='channel_id')
merge_df.head()

,video_id,channel_id,subscriberCount
0,s8XjEuplx_U,UC8butISFwT-Wl7EV0hUK0BQ,7890000
1,PXMJ6FS7llk,UC8butISFwT-Wl7EV0hUK0BQ,7890000
2,bsq2YY-XAEo,UC0xRMqPOyRNPTaL6BxhbCnQ,139000
3,KmqEyT7HznQ,UC0xRMqPOyRNPTaL6BxhbCnQ,139000
4,VRFfAeW30qE,UC0xRMqPOyRNPTaL6BxhbCnQ,139000


In [ ]:
extracted_df = merge_df[merge_df['subscriberCount'] < 100000]
extracted_df.head()

,video_id,channel_id,subscriberCount
27,7dYwASPbjoc,UChrXC0d5cyWdZFY8OGsmraw,19700
32,3hz-9hcf1PM,UC1uj19dZQXa17-SqNGpVNLw,35100
37,L26nKhwlePM,UCuD3lRkYg9h02U5_e9c09Kw,6660
38,uaIUOQ47eP8,UCEq0oQsS-voRnSWbcviIDGA,9140
39,SPf_nINsCfw,UCGFxqqRaM97VjM5ToRfkoFg,8760


In [ ]:
video_ids = extracted_df['video_id'].tolist()
video_ids

['7dYwASPbjoc',
 '3hz-9hcf1PM',
 'L26nKhwlePM',
 'uaIUOQ47eP8',
 'SPf_nINsCfw',
 'OMZwld35T4c',
 '_aVJH4JBWbE',
 'JP_ugJny9X4',
 'Eu3CojjLQL4',
 '2VkJhtDAKIc',
 'mWPI8vI2scQ',
 'nCfAtMooIQQ',
 'BXkDhnEBjgs',
 '8xFdQ-0t12s']

# 動画取得

In [ ]:
videos_list = youtube.videos().list(
    part='snippet,statistics',
    id=','.join(video_ids),
    fields='items(id,snippet(title),statistics(viewCount))',
).execute()

In [ ]:
videos = []
for item in videos_list['items']:
    video = {
        'video_id': item['id'],
        'title': item['snippet']['title'],
        'viewCount': int(item['statistics']['viewCount'])
    }
    videos.append(video)

videos_df = pd.DataFrame(videos)
videos_df.head()
    

,video_id,title,viewCount
0,7dYwASPbjoc,【エクセルスゴ技！】シフト表を自動化してみよう！,595619
1,3hz-9hcf1PM,【わりと有益】Pythonを習得して副業で稼ぐ方法5選,265351
2,L26nKhwlePM,用ChatGPT 撰寫操作Excel 的Python 程式，ChatGPT 能夠取代軟體工程...,101031
3,uaIUOQ47eP8,Google IT Automation with Python Professional ...,86322
4,SPf_nINsCfw,"Pythonで業務自動化(Excel,Outlook等)したいならこの本で決定",82833


In [ ]:
results = pd.merge(left=extracted_df, right=videos_df, on='video_id')
results.head()


,video_id,channel_id,subscriberCount,title,viewCount
0,7dYwASPbjoc,UChrXC0d5cyWdZFY8OGsmraw,19700,【エクセルスゴ技！】シフト表を自動化してみよう！,595619
1,3hz-9hcf1PM,UC1uj19dZQXa17-SqNGpVNLw,35100,【わりと有益】Pythonを習得して副業で稼ぐ方法5選,265351
2,L26nKhwlePM,UCuD3lRkYg9h02U5_e9c09Kw,6660,用ChatGPT 撰寫操作Excel 的Python 程式，ChatGPT 能夠取代軟體工程...,101031
3,uaIUOQ47eP8,UCEq0oQsS-voRnSWbcviIDGA,9140,Google IT Automation with Python Professional ...,86322
4,SPf_nINsCfw,UCGFxqqRaM97VjM5ToRfkoFg,8760,"Pythonで業務自動化(Excel,Outlook等)したいならこの本で決定",82833


In [ ]:
resutls = results.loc[:, ['video_id', 'title', 'viewCount', 'subscriberCount', 'channel_id']]
results.head()

,video_id,channel_id,subscriberCount,title,viewCount
0,7dYwASPbjoc,UChrXC0d5cyWdZFY8OGsmraw,19700,【エクセルスゴ技！】シフト表を自動化してみよう！,595619
1,3hz-9hcf1PM,UC1uj19dZQXa17-SqNGpVNLw,35100,【わりと有益】Pythonを習得して副業で稼ぐ方法5選,265351
2,L26nKhwlePM,UCuD3lRkYg9h02U5_e9c09Kw,6660,用ChatGPT 撰寫操作Excel 的Python 程式，ChatGPT 能夠取代軟體工程...,101031
3,uaIUOQ47eP8,UCEq0oQsS-voRnSWbcviIDGA,9140,Google IT Automation with Python Professional ...,86322
4,SPf_nINsCfw,UCGFxqqRaM97VjM5ToRfkoFg,8760,"Pythonで業務自動化(Excel,Outlook等)したいならこの本で決定",82833


In [ ]:
class Youtube:
    def __init__(self, service: str, version: str, developerkey: str) -> None:
        self.app = build(service, version, developerKey=developerkey)
    
    def search(self, keyword: str, max_results: int=50) -> pd.DataFrame:
        search_response = self.app.search().list(
            q=keyword,
            part="id,snippet",
            order='viewCount',
            type='video',
            maxResults=max_results
        ).execute()

        items_id = []
        for item in search_response['items']:
            items_id.append({
                'video_id': item['id']['videoId'], 
                'channel_id': item['snippet']['channelId']
                })

        df = pd.DataFrame(items_id)
        return df
    
    def get_channel_statistics(self, 
                               channelds: list | pd.DataFrame, 
                               max_results: int=50) -> pd.DataFrame:
        
        if isinstance(channelds, pd.DataFrame):
            channles_id = channelds['channel_id'].unique().tolist()
        else:
            channles_id = channelds

        statistics_list = self.app.channels().list(
            id=','.join(channles_id),
            part='statistics',
            fields='items(id, statistics(subscriberCount))',
            maxResults=max_results
        ).execute()
        
        statistics = []
        for item in statistics_list['items']:
            if len(item['statistics']) > 0:
                statistic = {
                    'channel_id': item['id'],
                    'subscriberCount': int(item['statistics']['subscriberCount'])
                }
            else:
                statistic = {
                    'channel_id': item['id'],
                }
            statistics.append(statistic)

        df = pd.DataFrame(statistics).fillna(0.0)
        return df

    def get_extracted_videos(self, df: pd.DataFrame, subscribercount: int, 
                             mt: bool=False, lt: bool=False) -> pd.DataFrame:
        if subscribercount:
            if mt:
                extracted_df = df[df['subscriberCount'] >= subscribercount]
            else:
                extracted_df = df[df['subscriberCount'] <= subscribercount]
        return extracted_df
    
    def get_videos(self, 
                   videos: list | pd.DataFrame, 
                   max_results: int=50) -> pd.DataFrame:
        
        if isinstance(videos, pd.DataFrame):
            video_ids = videos['video_id'].tolist()
        else:
            video_ids = videos
        videos_list = self.app.videos().list(
            part='snippet,statistics',
            id=','.join(video_ids),
            fields='items(id,snippet(title),statistics(viewCount))',
            maxResults=max_results
        ).execute()

        videos = []
        for item in videos_list['items']:
            video = {
                'video_id': item['id'],
                'title': item['snippet']['title'],
                'viewCount': int(item['statistics']['viewCount'])
            }
            videos.append(video)

        df = pd.DataFrame(videos)
        return df
    
    def get(self, keyword: str, subscribercount: int, mt: bool=False, lt: bool=False, 
            max_results=50):
        # 動画検索
        search_df = self.search(keyword, max_results)
        # 統計情報取得
        statistics_df = self.get_channel_statistics(search_df)
        # マージ
        merge_df = pd.merge(left=search_df, right=statistics_df, on='channel_id')
        # フィルター
        extracted_df = self.get_extracted_videos(
            merge_df, subscribercount=subscribercount, mt=mt, lt=lt)
        
        # 動画詳細情報取得
        video_df = self.get_videos(extracted_df)
        # マージ
        merge_df = pd.merge(left=extracted_df, right=video_df, on='video_id')
        # 列順整理
        resutls = merge_df.loc[:, ['video_id', 'title', 'viewCount', 'subscriberCount', 'channel_id']]
        return resutls